In [1]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime
import time, schedule
import numpy as np

In [2]:
def descargar_xml(url, nombre_archivo):
    try:
        respuesta = requests.get(url)
        respuesta.raise_for_status()  # Lanza un error si la solicitud falla
        
        with open(nombre_archivo, 'wb') as archivo:
            archivo.write(respuesta.content)
        
        print(f"Archivo XML guardado como {nombre_archivo}")
    except requests.exceptions.RequestException as e:
        print(f"Error al descargar el archivo: {e}")

In [5]:
url = "https://datos.madrid.es/egob/catalogo/202087-0-trafico-intensidad.xml"
filepath = "./data/trafico_intensidad.xml"
# Descargar el archivo XML
descargar_xml(url, filepath)

Archivo XML guardado como ./data/trafico_intensidad.xml


In [6]:
def xml_to_df(filepath):
    """read xml file and convert to pandas dataframe
       The xml fomrat is:
       <pms>
            <fecha_hora>01/04/2025 14:40:03</fecha_hora>
            <pm>
                <idelem>3840</idelem>
                <descripcion>Jose Ortega y Gasset E-O - Pº Castellana-Serrano</descripcion>
                <accesoAsociado>0512003</accesoAsociado>
                <intensidad>140</intensidad>
                <ocupacion>2</ocupacion>
                <carga>22</carga>
                <nivelServicio>0</nivelServicio>
                <intensidadSat>830</intensidadSat>
                <error>S</error>
                <subarea>0101</subarea>
                <st_x>441617,090594015</st_x>
                <st_y>4475765,7692796</st_y>
            </pm>
            ...
       The target df have to store the following columns:
       ['id', 'fecha', 'intensidad', 'ocupacion', 'carga', 'vmed']
    """
    tree = ET.parse(filepath)
    root = tree.getroot()

    # Find the global fecha_hora value
    fecha_value = root.find('fecha_hora')
    fecha_value = fecha_value.text if fecha_value is not None else np.nan

    # Crear listas para almacenar los datos
    ids = []
    fechas = []
    intensidades = []
    ocupaciones = []
    cargas = []
    vmeds = []

    # Iterar sobre los elementos del XML y extraer los datos
    for pm in root.findall('pm'):
        idelem = pm.find('idelem')
        intensidad = pm.find('intensidad')
        ocupacion = pm.find('ocupacion')
        carga = pm.find('carga')
        intensidadSat = pm.find('intensidadSat')

        ids.append(idelem.text if idelem is not None and idelem.text is not None else np.nan)
        fechas.append(fecha_value)
        intensidades.append(intensidad.text if intensidad is not None and intensidad.text is not None else np.nan)
        ocupaciones.append(ocupacion.text if ocupacion is not None and ocupacion.text is not None else np.nan)
        cargas.append(carga.text if carga is not None and carga.text is not None else np.nan)
        vmeds.append(intensidadSat.text if intensidadSat is not None and intensidadSat.text is not None else np.nan)

    # Crear un DataFrame de pandas
    df = pd.DataFrame({
        'id': ids,
        'fecha': fechas,
        'intensidad': intensidades,
        'ocupacion': ocupaciones,
        'carga': cargas,
        'vmed': vmeds
    })

    # Convertir la columna 'fecha' a tipo datetime, si no es NaN
    df['fecha'] = pd.to_datetime(df['fecha'], format='%d/%m/%Y %H:%M:%S', errors='coerce')
    df['id'] = pd.to_numeric(df['id'], errors='coerce')

    return df


In [7]:
df = xml_to_df(filepath)
df.head()

,id,fecha,intensidad,ocupacion,carga,vmed
0,3840,2025-04-15 15:30:04,220,25,26,830
1,3841,2025-04-15 15:30:04,280,3,17,1800
2,3842,2025-04-15 15:30:04,1240,8,47,2500
3,3843,2025-04-15 15:30:04,660,2,21,3500
4,3844,2025-04-15 15:30:04,920,3,26,4050


In [8]:
# Count rows
row_count = df.shape[0]
print(f"Number of rows: {row_count}")

# count nans per column
nan_counts = df.isna().sum()
print("NaN counts per column:")
print(nan_counts)

# Filter stations of interest
stations_of_interest = [3492, 3493, 6791, 6792, 3600, 3838, 10178, 10179, 6753, 6754,6130, 6131, 3781, 10202, 10203, 10204,6444, 6449, 10353, 6445,6505, 6506, 6507]
df_filtered = df[df['id'].isin(stations_of_interest)]
# Count rows after filtering
row_count_filtered = df_filtered.shape[0]
print(f"Number of rows after filtering: {row_count_filtered}")

Number of rows: 5328
NaN counts per column:
id              0
fecha           0
intensidad      0
ocupacion       0
carga           0
vmed          310
dtype: int64
Number of rows after filtering: 23


In [9]:
df_filtered


,id,fecha,intensidad,ocupacion,carga,vmed
3963,3781,2025-04-15 15:30:04,80,3,13,1800
3965,6130,2025-04-15 15:30:04,260,68,44,1800
3966,6131,2025-04-15 15:30:04,220,73,36,1800
4476,6444,2025-04-15 15:30:04,600,26,48,1200
4477,6445,2025-04-15 15:30:04,500,7,40,1200
4481,6449,2025-04-15 15:30:04,640,7,35,1650
4488,10353,2025-04-15 15:30:04,440,6,21,1800
4968,6506,2025-04-15 15:30:04,660,3,51,1125
4970,6505,2025-04-15 15:30:04,720,5,20,3600
4974,6507,2025-04-15 15:30:04,360,32,31,1325
